# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846845331971                   -0.70    4.0
  2   -7.852322898086       -2.26       -1.53    1.0
  3   -7.852646081198       -3.49       -2.52    3.2
  4   -7.852646678508       -6.22       -3.34    2.2
  5   -7.852646686049       -8.12       -4.80    1.0
  6   -7.852646686725       -9.17       -5.24    4.2
  7   -7.852646686730      -11.38       -5.99    1.8
  8   -7.852646686730      -12.60       -7.36    2.0
  9   -7.852646686730      -14.45       -7.59    2.5
 10   -7.852646686730   +  -14.75       -8.61    2.5
 11   -7.852646686730   +    -Inf       -9.24    2.0
 12   -7.852646686730   +    -Inf      -10.52    2.0
 13   -7.852646686730   +    -Inf      -10.81    2.2
 14   -7.852646686730   +    -Inf      -11.97    2.0
 15   -7.852646686730      -14.75      -12.86    2.2


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846816619553                   -0.70           4.0
  2   -7.852527051283       -2.24       -1.62   0.80    5.0
  3   -7.852608327609       -4.09       -2.79   0.80    5.2
  4   -7.852646568926       -4.42       -3.55   0.80    7.2
  5   -7.852646683693       -6.94       -4.49   0.80    8.8
  6   -7.852646686670       -8.53       -5.12   0.80   10.2
  7   -7.852646686725      -10.26       -6.28   0.80   12.0
  8   -7.852646686730      -11.33       -7.48   0.80   13.8
  9   -7.852646686730      -13.60       -7.71   0.80   15.8
 10   -7.852646686730   +  -14.45       -9.02   0.80   15.8
 11   -7.852646686730      -14.75       -9.69   0.80   18.0
 12   -7.852646686730   +    -Inf      -10.90   0.80   19.8
 13   -7.852646686730   +  -14.75      -10.99   0.80   22.5
 14   -7.852646686730      -14.15      -12.10   0.80   22.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.403830e+01     3.551097e+00
 * time: 0.5489048957824707
     1     1.254892e+00     1.848070e+00
 * time: 0.811784029006958
     2    -1.523633e+00     2.137760e+00
 * time: 0.8423030376434326
     3    -3.749176e+00     1.847560e+00
 * time: 0.8857250213623047
     4    -5.059144e+00     1.555974e+00
 * time: 0.9263899326324463
     5    -6.829387e+00     7.953645e-01
 * time: 0.9666829109191895
     6    -7.457615e+00     2.966669e-01
 * time: 1.0084960460662842
     7    -7.633320e+00     1.869104e-01
 * time: 1.0367748737335205
     8    -7.727485e+00     1.611236e-01
 * time: 1.06510591506958
     9    -7.788159e+00     8.201453e-02
 * time: 1.09397292137146
    10    -7.827605e+00     6.465391e-02
 * time: 1.2369909286499023
    11    -7.840488e+00     7.121719e-02
 * time: 1.2668418884277344
    12    -7.845426e+00     4.007867e-02
 * time: 1.2950809001922607
    13    -7.846289e+00     4.202238e-02
 * time: 1.325314044952392

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846809534354                   -0.70    4.0
  2   -7.852291427937       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646685966                   -2.55
  2   -7.852646686730       -9.12       -5.56
  3   -7.852646686730   +    -Inf      -10.40
  4   -7.852646686730   +    -Inf      -15.95
|ρ_newton - ρ_scf|  = 1.2022676244115545e-13
|ρ_newton - ρ_scfv| = 1.7385342567430597e-12
|ρ_newton - ρ_dm|   = 2.15646223989216e-10
